In [1]:


from Lab28_session_header import *

# Please update these path varibles to your local machine.
env_session_file_location = r"\\172.24.54.234\\NAS-Lab28\\Data"
NAS_Path = r"\\172.24.54.234\\NAS-Lab28"
Plot_path= r"\\172.24.54.234\\NAS-Lab28\\Users\\Bane\\Plots"

#Bg database
bgdb=pd.read_csv(NAS_Path+"\\DataBases\\bg_db.csv")


# Session Analysis

## Build Run list from new database

In [2]:
Connection = sqlite3.connect(NAS_Path+"\\Data\\lab_28_run.db")
RunDatabase = pd.read_sql("SELECT * from runs", Connection).sort_values(by='run', ascending=False)
Connection.close()


In [3]:

def Get_Waveform_from_session(run_number: int):
    """
    Get_Waveform_from_session(run_number: int) -> dict
    This function takes a run number and returns a dictionary of the waveforms from the session file."""
    RunInfo = RunDatabase.query(f'run == {run_number}').iloc[0]
    session_file_name= f"{env_session_file_location}\\\{RunInfo['session']:03}_{RunInfo['date']}.h5"
    run_dict={}
    with h5py.File(session_file_name, 'r') as session_h5:
        keys = list(session_h5["run"][str(run_number)].keys())
        for  key in keys:
            run_dict[key] = session_h5['run'][str(run_number)][key][:]
    return run_dict    

def GetBGRun(run):
    global bgdb
    try:
        return bgdb.query("run==%i"%(run)).iloc[0]["background"]
    except:
        return -1



In [4]:
def GetSinBG(x_data,y_data,time_start=0,time_stop=10,stepsize=100,debug=10,plot=True):
    
    x_fit=[]
    y_fit=[]
    
    if np.mean(y_data)*1000 <= 0.001:
        print("issue with weird data")
        return [0,0,0,0,0],[0,0,0,0,0]

    if len(x_data) <=1 or len(y_data)<=1 or len(x_data) != len(y_data):

        print("X and Y need to be equal in length and need two more data points")
        print(" X: ", len(x_data), " Y :" ,len(y_data))
        return np.zeros(len(x_data)),np.zeros(len(y_data))
    
    if plot: sin_figure, sin_ax =plt.subplots(figsize=(9,4))


    # Determine fitting limits
    time_resolution= x_data[1] - x_data[0]
    index_start = int(time_start/time_resolution)
    index_stop = int(time_stop/time_resolution)
    if debug>=10: print("Start ",time_start,index_start," stop ",time_stop,index_stop)
    
    if plot:
        sin_ax.errorbar(x_data,y_data,alpha=0.75,
    linestyle=(0, (5, 1)),c="gray",label="Data")
        sin_ax.legend()

    # smooth out the data
    smooth_slice=slice(int(stepsize/2),-(int(stepsize/2)+1))
    smooth_ydata=np.convolve(y_data,
     np.ones(stepsize)/stepsize,mode="same")[smooth_slice]
    smooth_xdata=x_data[smooth_slice]
    if plot:sin_ax.errorbar(smooth_xdata,smooth_ydata,
    linestyle=(0, (5, 1)),label="Smooth Data")

    # Prepare for fit
    x_fitdata=smooth_xdata[index_start:index_stop-int(stepsize/2)]
    y_fitdata=smooth_ydata[index_start:index_stop-int(stepsize/2)]
    #if plot:sin_ax.errorbar(x_fitdata,y_fitdata,fmt=":",label="Fit Data")

    ## fit the expontial tail first 
    if y_fitdata[0] < y_fitdata[-1]:
        print("Weird structure skipping sin fit")
        return [0,0,0,0,0],[0,0,0,0,0]

    p0_tail_only = [np.max(y_fitdata),-0.001]
    bounds_tail_only = [np.max(y_fitdata)*.75,-0.01],[np.max(y_fitdata)*1.25,0.0001]
    if debug >=10:
        print("Tail Only")
        print("P0", p0_tail_only)
        print("bounds",bounds_tail_only)

    params_tail_only, covariance_tail_only = curve_fit(simple_exp, 
        x_fitdata, y_fitdata,p0=p0_tail_only,bounds=bounds_tail_only)


    x_fit=smooth_xdata
    y_fit_tail_only=simple_exp(x_fit,*params_tail_only)
    if plot:
        sin_ax.errorbar(x_fit,y_fit_tail_only,
        linestyle=":",label="Only tail")
        

    if debug>=5: print("Only exp tail paramaters", params_tail_only)

    # subtracted the tail

    tail_sub_fitdata = y_fitdata - simple_exp(x_fitdata,*params_tail_only)

    
    #po = amp,freq, phase, exp amp, decay
    #p0=[max(y_fitdata)/2-min(y_fitdata)/2,0.25,0.5,params_tail_only[0],params_tail_only[1]]
    #bounds=[0,0.0001,-np.pi,params_tail_only[0]*0.8,params_tail_only[1]*1.2],[p0[0],1.0,np.pi,params_tail_only[0]*1.2,params_tail_only[1]*0.8]

    # po for freq. try to count number of local max for time range

    # amp, freq, phase,
    p0 = [(np.max(tail_sub_fitdata) - np.min(tail_sub_fitdata))/2, 0.14,0 ]
    bounds =[0,0,-3.14 ],[((np.max(tail_sub_fitdata) - np.min(tail_sub_fitdata))/2 )*1.3,
            1,3.14]


    if debug>=5:
        print("Fit P0",p0)
        print("Bounds",bounds)
    params, covariance = [0,0,0],[[0,0,0],[0,0,0],[0,0,0]]
    try: 
        params, covariance = curve_fit(sin, 
            x_fitdata, tail_sub_fitdata,p0=p0,bounds=bounds)
    except:
        print("Issue with sin fit")

    if debug >=5:
        print("Parameters", np.round(params,3))
    if debug>=10: print("Covariance",covariance)
    
    x_fit=smooth_xdata
    y_fit=sin(x_fit,params[0],params[1],params[2])
    if plot:
        sin_ax.errorbar(x_fitdata,sin_exp(x_fitdata,params[0],params[1],params[2],
            params_tail_only[0], params_tail_only[1]),
        linestyle="--",label="Fit Results")
        sin_ax.errorbar(x_fit,y_fit+params_tail_only[0],linestyle="--",label="Sin Fit")
        sin_ax.errorbar(x_fit[::stepsize],(smooth_ydata-y_fit)[::stepsize],
                        color="black",fmt="-.",linewidth=0.75,label="Subtracted")

    y_fit_Stddev=sin_exp(x_fitdata,params[0],params[1],params[2],
            params_tail_only[0], params_tail_only[1])
    dev = np.sqrt(np.sum(abs(y_fit_Stddev-y_fitdata)**2)/len(y_fit))
    if debug >=5: print("Fit deviation from fitting data", dev)
    if dev>1:
        print("\t\t!!!!!!WARNING!!!!!!!!!  Issue with sin fit Large Deviation")
    #print((y_fit_Stddev-y_fitdata)/y_fitdata )
    if plot:
        sin_ax.grid()
        sin_ax.legend()
        #plt.xlim(time_start-12,time_stop-8)
        #plt.ylim(params[4] + params[3]*time_stop,params[4]*1.1)

    return [params[0],params[1],params[2],
            params_tail_only[0], params_tail_only[1]],[covariance[0][0],covariance[1][1],covariance[2][2],covariance_tail_only[0][0],covariance_tail_only[1][1]]

In [5]:
def Run_oldtonew(run):
    """
    Run_oldtonew(run) -> int
    This function takes a run number and returns the new run number."""
    global RunDatabase
    try:
        return RunDatabase[RunDatabase['old_run_name'].str.contains(f'{run:05}')].iloc[0]['run']
    except:
        print("No old run number found")
        return -1
    

def Run_newtoold(run):
    """
    Run_newtoold(run) -> int
    This function takes a run number and returns the old run number."""
    global RunDatabase
    try:
        old_run_name=RunDatabase[RunDatabase['run'] == run].iloc[0]['old_run_name']
    except:
        print("No new run number found")
        return -1
    
    
    split =old_run_name.split("-")
    return int(split[1])

In [ ]:
stepcounts=[100]
stepcount=200
diffsizes=[200]
debug=4
plot=True
savefig=True

good_runs=[378,391,800,993]
run_number=good_runs[1]




waveform = Get_Waveform_from_session(run_number)

old_rn = Run_newtoold(run_number)
bgnum = Run_oldtonew(GetBGRun(old_rn))
background_waveform = Get_Waveform_from_session(bgnum)



def Analyze_Anode(waveform,background_wavefrom, stepcounts=[100], diffsizes=[100],debug=0,plot=False,savefig=True):


    if debug>=1:
        print("::Anode Analysis::")
        print("Run Number",run_number)
        print("Background Run Number",bgnum)

    AnalysisResults={}

    last_index= np.min([len(waveform["time"]),len(waveform["anode"]),len(background_waveform["anode"]),len(background_waveform["time"])])
    
    waveform["time"] = waveform["time"][:last_index]
    waveform["anode"] = waveform["anode"][:last_index]
    background_waveform["time"] = background_waveform["time"][:last_index]
    background_waveform["anode"] = background_waveform["anode"][:last_index]
    
    waveform["time"] = waveform["time"]/1e3 #convert to us
    background_waveform["anode"] = background_waveform["anode"]*1e3 #convert to mv
    waveform["anode"] = waveform["anode"]*1e3 #convert to mv

    time_resolution = waveform["time"][1] - waveform["time"][0]
    #Find the average anode for the first 5 us and subtract to normalize. 
    anode_normalize = waveform["anode"][:int(5/time_resolution)].mean()

    waveform["norm"] = {}
    wavefrom["denoise"] = {}
    waveform["bg_sub"] = {}
    
    waveform["norm"]["anode"] = waveform["anode"] - anode_normalize
    ``    

    #Find the average anode for the first 5 us and subtract to normalize.    
    background_waveform["anode_norm"] =background_waveform["anode"]- background_waveform["anode"][:int(5/time_resolution)].mean()
    
    edge_slice=slice(0,int(45/time_resolution))

    

    if plot:
        fig_anode, ax_anode = plt.subplots(figsize=(9,5))
        ax_anode.grid()
        ax_anode.set_xlabel("time [us]")
        ax_anode.set_ylabel("Signal [mV]")

        ax_anode.errorbar(x=waveform["time"][edge_slice],y=waveform["anode_norm"][edge_slice], label="Normalized Signal")
        ax_anode.errorbar(x=waveform["time"][edge_slice],y=background_waveform["anode_norm"][edge_slice], label="Back Ground")


    #Find and remove noise
    Anode_noise_params, Anode_noise_cov =GetSinBG(waveform["time"],waveform["anode_norm"],27.5,50,debug=debug,plot=plot)
    BG_noise_params, BG_noise_cov = GetSinBG(waveform["time"],background_waveform["anode_norm"],25.5,50,150,debug=debug,plot=plot)
                                             
    noise_waveform = sin(waveform["time"],Anode_noise_params[0],Anode_noise_params[1],Anode_noise_params[2])
    bg_noise_waveform = sin(background_waveform["time"],BG_noise_params[0],BG_noise_params[1],BG_noise_params[2])
    

    #subtract the noise from the anode signal
    waveform["anode_noise_subtracted"] = waveform["anode_norm"] - noise_waveform
    background_waveform["anode_noise_subtracted"] = background_waveform["anode_norm"] - bg_noise_waveform     


    waveform["anode_noise_bg_subtracted"] = waveform["anode_noise_subtracted"] - background_waveform["anode_noise_subtracted"]
    waveform["anode_bg_subtracted"] = waveform["anode_norm"] - background_waveform["anode_norm"]
    waveform["anode_final"]=np.convolve(waveform["anode_noise_bg_subtracted"],np.ones(stepcount)/stepcount,mode="same")
    waveform["anode_norm_background"] = background_waveform["anode_norm"]

    if plot:
        ax_anode.errorbar(x=waveform["time"][edge_slice],y=waveform["anode_noise_subtracted"][edge_slice], label="Signal - Noise")
        ax_anode.errorbar(x=waveform["time"][edge_slice],y=background_waveform["anode_noise_subtracted"][edge_slice],color="grey", alpha=1, label="BG - Noise")
        ax_anode.errorbar(x=waveform["time"][edge_slice],y=waveform["anode_noise_bg_subtracted"][edge_slice],color="black", alpha=1, label="Signal - BG - Noise")        
        ax_anode.errorbar(x=waveform["time"][edge_slice],y=waveform["anode_final"][edge_slice],color="red", alpha=1, label="Signal - BG - Noise - Smooth")
        

    waveform["anode_derivative"] = np.gradient(waveform["anode_final"],2/time_resolution)

    if plot:
        fig_anode_der, ax_der = plt.subplots(figsize=(9,5))
        ax_der.grid()
        ax_der.set_xlabel("time [us]")
        ax_der.set_ylabel("Signal Derivative [mV]/[us]")
        ax_der.errorbar(x=waveform["time"][edge_slice],y=waveform["anode_derivative"][edge_slice], label="Derivative of Signal")

    # Fit derivative to find the peak and sigma for timeing of anode signal
    offset = int(np.where(abs(waveform["time"]-0.001)<=0.001)[0][0] + 5/time_resolution)


    Amp_p0  = np.max(waveform["anode_derivative"][offset:])
    peak_p0 = waveform["time"][np.argmax(waveform["anode_derivative"][offset:])+offset]
    sigma_p0 = 0.1
    p0 = [Amp_p0,peak_p0,sigma_p0]
    params, covariance = curve_fit(gaussian, waveform["time"], waveform["anode_derivative"],p0=p0)
    fit_y = gaussian(waveform["time"],*params)
    if debug>=4:
        print("Gaussian Fit Parameters",params)
        print("p0",p0)
    sigma_mod=3

    edge_location_2 = [int(np.where(fit_y == max(fit_y))[0][0] - sigma_mod*params[2]/(time_resolution)),
    int(np.where(fit_y == max(fit_y))[0][0] + sigma_mod*params[2]/time_resolution)]

    edge_location_1 =np.where(abs(fit_y -0.01*Amp_p0) <= 0.00025*Amp_p0)
    if debug>=10:
        print("Edge Location 1",edge_location_1)
        print("Edge Location 2",edge_location_2)

    Anode_final_low = waveform["anode_final"][edge_location_2[0]]
    Anode_final_high = waveform["anode_final"][edge_location_2[1]]
    Anode_final_amp = (Anode_final_high - Anode_final_low)
    Anode_final_avg_amp = (Anode_final_high + Anode_final_low)/2

    if debug>=10:print("Anode Final Amp", Anode_final_amp)



    if plot:
        ax_der.errorbar(x=waveform["time"][edge_slice],y=fit_y[edge_slice], label="Gaussian Fit")

        ax_der.errorbar(x=waveform["time"][edge_location_1],y=fit_y[edge_location_1], label="edge location", fmt="o", color="red")
        ax_der.errorbar(x=waveform["time"][edge_location_2],y=fit_y[edge_location_2], label="edge location2", fmt="x", color="blue")
        ax_anode.errorbar(x=waveform["time"][edge_location_2],y=waveform["anode_final"][edge_location_2], 
        label="edge location2", fmt="+", markersize=15, color="cyan")
        ax_anode.text(x=params[1]+3,y=Anode_final_avg_amp,s=f"Amp: {Anode_final_amp:.2f} mV")
        ax_anode.text(x=params[1]+3,y=Anode_final_avg_amp*0.8,s=f"Time Delay: {params[1]:.2f} us")
        ax_der.text(x=params[1]+2,y=params[0]*0.5,s=f"Time Delay: {params[1]:.2f}")

    AnalysisResults["Anode Time Delay"] = params[1]
    AnalysisResults["Anode Time Delay Sigma"] = params[2]
    AnalysisResults["Anode Signal Amp"] = Anode_final_amp
    AnalysisResults["Anode Noise Frq"]=Anode_noise_params[1]
    AnalysisResults["Anode Noise Amp"]=Anode_noise_params[0]
    AnalysisResults["Anode Signal Decay Rate"]=Anode_noise_params[4]
    AnalysisResults["BG Noise Frq"]=BG_noise_params[1]
    AnalysisResults["BG Noise Amp"]=BG_noise_params[0]
    AnalysisResults["BG Signal Decay Rate"]=BG_noise_params[4]


    if plot:
        ax_anode.legend()
    if debug>=5:
        print("Finishing Anode Analysis")

    print(AnalysisResults)


In [123]:
AAA={}
AAA["A"]={}
AAA["A"]["A"]={}
AAA["A"]["A"]["A"]=1


In [124]:
AAA["A"]["A"]["A"]

1

In [105]:
run_number=800
waveform = Get_Waveform_from_session(run_number)
old_rn = Run_newtoold(run_number)
bgnum = Run_oldtonew(GetBGRun(old_rn))
background_waveform = Get_Waveform_from_session(bgnum)


Analyze_Anode(waveform,background_waveform,debug=1,plot=False)



::Anode Analysis::
Run Number 800
Background Run Number 806
{'Anode Time Delay': 14.441966541682769, 'Anode Time Delay Sigma': 0.39401444914729694, 'Anode Signal Amp': 17.55684042692185, 'Anode Noise Frq': 0.13551025715761408, 'Anode Noise Amp': 0.020776768492334386, 'Anode Signal Decay Rate': -0.007170628038551859, 'BG Noise Frq': 0.15287139100172814, 'BG Noise Amp': 0.14452481345031912, 'BG Signal Decay Rate': 9.999999999999903e-05}


In [ ]:
## analysis file name - session_date-analysis_verision.h5
## example file name - 001_230202_analysis_01.h5



In [118]:
run_number=500
RunInfo = RunDatabase.query(f'run == {run_number}').iloc[0]

analysis_file_name = f"{env_session_file_location}\\\\analysis_trial\\\{RunInfo['session']:03}_{RunInfo['date']}_analysis_01.h5"

print("Analysis File Name",analysis_file_name)

with h5py.File(analysis_file_name, 'a') as analysis_h5:

    print("Creating Analysis File or appending old file")
    if "run" not in analysis_h5.keys():
        analysis_h5.create_group("run")
    print("Creating Run number")
    if str(run_number) not in analysis_h5["run"].keys():
        analysis_h5.create_group(f"run/{run_number}")

    for anlysis_key in ["norm","denoised","bg_subtracted","bg_subtracted_smoothed"]:
        if anlysis_key not in analysis_h5["run"][str(run_number)].keys():
            analysis_h5.create_group(f"run/{run_number}/{anlysis_key}")
        else:
            print("Key already exists, skipping")
    
    # Save time, trigger, and UV data to the analysis file
    analysis_h5["run"][str(run_number)].create_dataset("time", data=waveform["time"])
    analysis_h5["run"][str(run_number)].create_dataset("trigger", data=waveform["trigger"])
    analysis_h5["run"][str(run_number)].create_dataset("uv", data=waveform["uv"])

    



Analysis File Name \\172.24.54.234\\NAS-Lab28\\Data\\analysis_trial\\024_241030_analysis_01.h5
Creating Analysis File or appending old file
Creating Run number
Key already exists, skipping
Key already exists, skipping
Key already exists, skipping
Key already exists, skipping


In [102]:
waveform

{'anode': array([30.62063 , 30.54096 , 30.937239, ..., 46.894318, 46.99971 ,
        46.65688 ], dtype=float32),
 'cathode': array([0.03439386, 0.03502921, 0.03529405, ..., 0.00701054, 0.00656055,
        0.00740701], dtype=float32),
 'time': array([-5.003, -4.999, -4.995, ..., 44.989, 44.993, 44.997], dtype=float32),
 'trigger': array([ 0.        ,  0.        , -0.03952387, ..., -0.03952387,
         0.        , -0.03952387], dtype=float32),
 'uv': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 'anode_norm': array([-0.08658791, -0.16625786,  0.23002052, ..., 16.1871    ,
        16.292492  , 15.949661  ], dtype=float32),
 'anode_noise_subtracted': array([-0.06788913, -0.14759004,  0.24865717, ..., 16.19891   ,
        16.30436   , 15.961587  ], dtype=float32),
 'anode_noise_bg_subtracted': array([-0.5110024 ,  0.5224909 ,  0.16366358, ..., 13.875694  ,
        13.800572  , 13.87091   ], dtype=float32),
 'anode_bg_subtracted': array([-0.40651512,  0.45832443,  0.091362  , ..., 1

In [68]:
Get_Waveform_from_session(580)


{'anode': array([0.02666183, 0.02664507, 0.02674159, ..., 0.02994496, 0.02997418,
        0.0297816 ], dtype=float32),
 'cathode': array([0.05593002, 0.0559302 , 0.05607319, ..., 0.00860732, 0.00859894,
        0.00838898], dtype=float32),
 'time': array([-10004., -10000.,  -9996., ...,  39988.,  39992.,  39996.],
       dtype=float32),
 'trigger': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 'uv': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)}

In [32]:
session_files = glob.glob(env_session_file_location + "\\*.h5")

for session_file in session_files:
    # Open the HDF5 file
    with h5py.File(session_file, 'r') as f:
        # Check if 'data' exists in the file
        subdir = f['run']
        for runnumber in subdir.keys():
            channels=subdir[runnumber].keys()
            waveform_data={}
            for channel in channels:
                waveform_data[channel] = subdir[runnumber][channel]
                print(subdir[runnumber][channel])

                print(subdir[runnumber][channel])
                print(waveform_data)
            break


    break


<HDF5 dataset "anode": shape (12501,), type "<f4">
<HDF5 dataset "anode": shape (12501,), type "<f4">
{'anode': <HDF5 dataset "anode": shape (12501,), type "<f4">}
<HDF5 dataset "cathode": shape (12501,), type "<f4">
<HDF5 dataset "cathode": shape (12501,), type "<f4">
{'anode': <HDF5 dataset "anode": shape (12501,), type "<f4">, 'cathode': <HDF5 dataset "cathode": shape (12501,), type "<f4">}
<HDF5 dataset "time": shape (12501,), type "<f4">
<HDF5 dataset "time": shape (12501,), type "<f4">
{'anode': <HDF5 dataset "anode": shape (12501,), type "<f4">, 'cathode': <HDF5 dataset "cathode": shape (12501,), type "<f4">, 'time': <HDF5 dataset "time": shape (12501,), type "<f4">}
<HDF5 dataset "trigger": shape (12501,), type "<f4">
<HDF5 dataset "trigger": shape (12501,), type "<f4">
{'anode': <HDF5 dataset "anode": shape (12501,), type "<f4">, 'cathode': <HDF5 dataset "cathode": shape (12501,), type "<f4">, 'time': <HDF5 dataset "time": shape (12501,), type "<f4">, 'trigger': <HDF5 dataset 

## Laser power analysis - calibration for laser settings to injected power


In [ ]:
runnumber = 500
debug = 10

#def MeasureLaserpower(runnumber=0, debug=0):
if 1==1:

    if debug >=10:
        print(f"Laser power analysis for Run number: {runnumber}")
    Runinfo = RunDatabase.query(f'run == {runnumber}').iloc[0]
    

    
